<a href="https://colab.research.google.com/github/rajkiran15/colab-hands-on/blob/main/rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from groq import Groq

# ---- 1. init embedding (local, free) ----
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def embed(text):
    embedded_text = embedder.encode(text)
    print(f"embedded_text {embedded_text}\n")

# ---- 2. sample documents (pretend chunks) ----
documents = [
    "LLM stands for Large Language Model.",
    "LLMs are neural networks trained on text to generate responses.",
    "RAG retrieves relevant text chunks, then LLM generates answers using that context.",
    "Cosine similarity finds the most relevant document chunks for a query.",
    "Prime Minister of India is Raj Kiran."
]

doc_embeddings = np.vstack([embed(doc) for doc in documents])
print(f"doc_embeddings: {doc_embeddings}\n")

# ---- 3. retriever top-k ----
def retrieve(query, k=2):
    q_vec = embed(query).reshape(1, -1)
    print(f"q_vec: {q_vec}\n")
    sims = cosine_similarity(q_vec, doc_embeddings)[0]
    print(f"sims: {sims}\n")
    top_idx = sims.argsort()[::-1][:k]
    print(f"top_idx: {top_idx}\n")
    return [documents[i] for i in top_idx]

# ---- 4. generator with real LLM (Groq llama 70B) ----
groq_client = Groq(api_key="")

def rag_answer(question, k=2):
    context = "\n".join(retrieve(question, k))
    print(f"context: {context}\n")

    prompt = f"""Answer using only context below.
                 If not present, say 'I don't know'.

                Context:
                {context}

                Question: {question}
                Answer in one line:
              """

    chat = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=100
    )
    return chat.choices[0].message.content

# ---- Demo ----
q = "Who is the pm of India?"
print("Used context:", retrieve(q))
print("Answer:", rag_answer(q))


embedded_text [ 4.15406339e-02 -1.01077087e-01  1.87924993e-03 -4.42914553e-02
  3.37561034e-02  3.02838013e-02 -6.05387129e-02  2.94945911e-02
  4.16090153e-02 -2.23719259e-03  2.54839212e-02  3.20125818e-02
  5.94761334e-02 -3.17548513e-02  1.56532358e-02  6.35513142e-02
  5.77049032e-02  2.05242429e-02  1.13346530e-02 -3.51956598e-02
  4.47167382e-02  5.88586703e-02  3.70776467e-02  3.73272002e-02
  3.67994867e-02  1.39915692e-02 -1.23628629e-02  4.93662544e-02
  6.82602823e-02 -8.73053819e-03 -9.19390097e-03  1.13159336e-01
  3.09079234e-02  6.42192662e-02 -4.16155495e-02  1.38687622e-02
 -6.33851886e-02 -3.43215652e-02 -1.65228657e-02 -2.21538637e-02
 -5.41482214e-03  4.79552383e-03  6.66231960e-02 -3.28120366e-02
  7.94580355e-02  3.33244316e-02 -2.73155794e-02  1.58389602e-02
 -1.67755529e-01  6.27115294e-02 -5.20665012e-03 -4.76479568e-02
 -5.78739345e-02  2.88844295e-02 -4.45740623e-03 -7.82380328e-02
  2.48867311e-02 -2.56453059e-03 -5.97167462e-02 -3.03293634e-02
 -5.6887779

AttributeError: 'NoneType' object has no attribute 'reshape'